In [34]:
import csv
import chromadb
import dotenv
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders.csv_loader import CSVLoader

In [41]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Step 1: ETL
- Read raw CSV data
- Parse raw data into Documents

In [40]:
loader = CSVLoader(file_path='./events.csv')
event_docs = loader.load()


### Init Persistant ChromaDB

In [42]:
persist_directory = 'datastore'
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=event_docs, embedding=embedding, persist_directory=persist_directory)